# **Starter Notebook to Combine the Image Captioning Network Output to the Image Generator Network to Evaluate Performance of Image Caption**   

Final Project - Mikayla Biggs, Kevin Steele, Austin Strom    
AML 4/26/2021
